# Your favourite notebooks sorted by date upvoted

Since there is currently no option to sort notebooks by date upvoted in the "Favorites" tab, I decided that it wouldn't take long for me to implement this using [Meta Kaggle](https://www.kaggle.com/kaggle/meta-kaggle).

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML, Markdown
        
KAGGLE_BASE_URL = "https://www.kaggle.com/"

# medal images
bronze = 'https://kaggle.com/static/images/medals/notebooks/bronzes@1x.png'
silver = 'https://kaggle.com/static/images/medals/notebooks/silvers@1x.png'
gold = 'https://kaggle.com/static/images/medals/notebooks/golds@1x.png'

# avatiers
novice = 'https://kaggle.com/static/images/avatier/avatier-novice@2x.png'
contributor = 'https://kaggle.com/static/images/avatier/avatier-contributor@2x.png'
expert = 'https://kaggle.com/static/images/avatier/avatier-expert@2x.png'
master = 'https://kaggle.com/static/images/avatier/avatier-master@2x.png'
grandmaster = 'https://kaggle.com/static/images/avatier/avatier-grandmaster@2x.png'
kaggle_staff = 'https://kaggle.com/static/images/avatier/avatier-staff@2x.png'

In [ ]:
kernels = pd.read_csv('/kaggle/input/meta-kaggle/Kernels.csv', 
                      parse_dates=['CreationDate', 'MadePublicDate', 'MedalAwardDate'],
                      infer_datetime_format=True
                     )
kernel_versions = pd.read_csv('/kaggle/input/meta-kaggle/KernelVersions.csv', 
                              parse_dates=['CreationDate'],
                              infer_datetime_format=True
                             )
kernel_votes = pd.read_csv('/kaggle/input/meta-kaggle/KernelVotes.csv', 
                           parse_dates=['VoteDate'], infer_datetime_format=True
                          )

users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv',
                    parse_dates=['RegisterDate'], 
                    infer_datetime_format=True
                   )

To use this notebook:
1. Fork this notebook by clicking on `Copy and Edit`.
1. Set the username variable to your own username.
1. Select sorting and filtering options below and run the notebook.

In [ ]:
username = 'sahidvelji'

MOST_RECENT_FIRST = True
INCLUDE_SELF_VOTES = True
NUM_RESULTS = 20

In [ ]:
userid = users.loc[users['UserName'].eq(username), 'Id'].item()
displayname = users.loc[users['UserName'].eq(username), 'DisplayName'].item()

df = (kernel_votes
      .loc[kernel_votes['UserId'].eq(userid), ['KernelVersionId', 'VoteDate']]
      .merge(kernel_versions[['Id', 'ScriptId']], left_on='KernelVersionId', right_on='Id')
      .merge(kernels[['Id', 'AuthorUserId', 'CurrentUrlSlug', 'CurrentKernelVersionId', 'Medal', 'TotalComments', 'TotalVotes']], left_on='ScriptId', right_on='Id')
      .merge(users, left_on='AuthorUserId', right_on='Id')
      .merge(kernel_versions[['Id', 'Title']], left_on='CurrentKernelVersionId', right_on='Id')
      .loc[:, ['VoteDate', 'CurrentUrlSlug', 'Title', 'UserName', 'DisplayName', 'PerformanceTier', 'Medal', 'TotalComments', 'TotalVotes']]
      .sort_values('VoteDate', ascending=not MOST_RECENT_FIRST)
     )

if not INCLUDE_SELF_VOTES:
    df = df[df['UserName'].ne(username)]
df = df.head(NUM_RESULTS)

df.loc[df['Medal'].eq(1), 'Medal'] = f'<img style="float:left;padding-right:2px" src="{gold}">'
df.loc[df['Medal'].eq(2), 'Medal'] = f'<img style="float:left;padding-right:2px" src="{silver}">'
df.loc[df['Medal'].eq(3), 'Medal'] = f'<img style="float:left;padding-right:2px" src="{bronze}">'
df['Medal'] = df['Medal'].fillna('')

df.loc[df['PerformanceTier'].eq(0), 'PerformanceTier'] = f'<img src="{novice}">'
df.loc[df['PerformanceTier'].eq(1), 'PerformanceTier'] = f'<img src="{contributor}">'
df.loc[df['PerformanceTier'].eq(2), 'PerformanceTier'] = f'<img src="{expert}">'
df.loc[df['PerformanceTier'].eq(3), 'PerformanceTier'] = f'<img src="{master}">'
df.loc[df['PerformanceTier'].eq(4), 'PerformanceTier'] = f'<img src="{grandmaster}">'
df.loc[df['PerformanceTier'].eq(5), 'PerformanceTier'] = f'<img src="{kaggle_staff}">'

df['URL'] = df['Medal'] + f'<a href="{KAGGLE_BASE_URL}' + df['UserName'] + '/' + df['CurrentUrlSlug'] + '">' + df['Title'] + '</a>'
df['User'] = f'<a href="{KAGGLE_BASE_URL}' + df['UserName'] + '">' + df['DisplayName'] + '</a>' + df['PerformanceTier']

columns = ['VoteDate', 'TotalVotes', 'User', 'URL', 'TotalComments']
df = df.reindex(columns=columns)

display(Markdown(f'# Notebooks upvoted by {displayname}'))

HTML(df
     .to_html(escape=False, index=False)
     .replace('<td>', '<td style="text-align:left">')
    )